# 🚀 Deploy Pipeline to Vertex AI - Example Template

This is an example template for deploying ML pipelines to Google Cloud Vertex AI.

**🔐 Setup Instructions:**

1. Copy this notebook and remove `.example` from the filename
2. Configure your GCP project using `config.py` or direct configuration
3. Complete the prerequisites below
4. Run the deployment cells


## Prerequisites

Before running this notebook, make sure:

1. ✅ Pipeline එක train කරලා `model.joblib` විදියට save කරලා තියෙන්න ඕනේ
2. ✅ Google Cloud Project එකක් හදලා billing enable කරන්න
3. ✅ Vertex AI API සහ Cloud Storage API enable කරන්න
4. ✅ Authentication කරගන්න: `gcloud auth application-default login`

### Enable Required APIs:

```bash
gcloud services enable aiplatform.googleapis.com
gcloud services enable storage.googleapis.com
```


In [ ]:
# Import Required Libraries
from google.cloud import aiplatform, storage
import os

print("✅ Libraries imported successfully!")

## Configuration

**Option 1: Use .env file (Recommended)**

1. Copy `.env.example` to `.env` in **this directory** (07_Pipeline_Deployment/)

2. Edit `.env` with your GCP project detailsFill in values directly in the second cell

3. Run the config cell below

**Option 2: Direct Configuration**


In [ ]:
# Option 1: Load from local config.py (which loads from .env)
try:
    from config import (GCP_PROJECT_ID, GCP_REGION, GCS_BUCKET_NAME,
                       ENDPOINT_DISPLAY_NAME, MODEL_DISPLAY_NAME, LOCAL_MODEL_PATH)

    PROJECT_ID = GCP_PROJECT_ID
    REGION = GCP_REGION
    BUCKET_NAME = GCS_BUCKET_NAME

    print("✅ Configuration loaded from config.py")
    print(f"Project: {PROJECT_ID}")
    print(f"Region: {REGION}")
    print(f"Bucket: gs://{BUCKET_NAME}")

except ImportError:
    print("⚠️ config.py not found in this directory.")
    print("Please create .env file (copy from .env.example) or use Option 2 below.")
    PROJECT_ID = None
    REGION = None
    BUCKET_NAME = None

    ENDPOINT_DISPLAY_NAME = None    LOCAL_MODEL_PATH = "model.joblib"
    MODEL_DISPLAY_NAME = None

In [ ]:
# Option 2: Direct Configuration (uncomment and fill in)

# PROJECT_ID = "your-project-id-here"  # Your GCP Project ID
# REGION = "us-central1"  # e.g., us-central1, asia-southeast1
# BUCKET_NAME = "your-bucket-name"  # GCS bucket name (WITHOUT gs:// prefix)
# MODEL_DISPLAY_NAME = "sentiment-pipeline-model"  # Name for your model
# ENDPOINT_DISPLAY_NAME = "sentiment-endpoint"  # Name for your endpoint
# LOCAL_MODEL_PATH = "model.joblib"  # The pipeline file

if 'LOCAL_MODEL_PATH' not in locals():
    LOCAL_MODEL_PATH = "model.joblib"

# Print configuration
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    print(f"✅ Project: {PROJECT_ID}")
    print(f"✅ Region: {REGION}")
    print(f"✅ Bucket: gs://{BUCKET_NAME}")
    print(f"✅ Model file: {LOCAL_MODEL_PATH}")
else:
    print("❌ Please configure PROJECT_ID, REGION, and BUCKET_NAME")

## Step 1: Create GCS Bucket (Optional)

Run this only if you need to create a new bucket:


In [ ]:
# Create GCS Bucket (uncomment if you need to create one)
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    # Uncomment the lines below to create a bucket
    # storage_client = storage.Client(project=PROJECT_ID)
    # try:
    #     bucket = storage_client.create_bucket(BUCKET_NAME, location=REGION)
    #     print(f"✅ Bucket {bucket.name} created in {REGION}")
    # except Exception as e:
    #     if "already exists" in str(e):
    #         print(f"✅ Bucket {BUCKET_NAME} already exists")
    #     else:
    #         print(f"❌ Error: {e}")
    print("💡 Bucket creation code is commented out. Uncomment if needed.")
else:
    print("❌ Please configure your project first")

## Step 2: Upload Pipeline to GCS

Upload your `model.joblib` (which contains the complete pipeline) to Google Cloud Storage:


In [ ]:
# Upload model.joblib to GCS bucket
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    try:
        storage_client = storage.Client(project=PROJECT_ID)
        bucket = storage_client.bucket(BUCKET_NAME)

        # Create a folder structure: model/model.joblib
        blob = bucket.blob("model/model.joblib")

        # Upload the file
        print("📤 Uploading pipeline to GCS...")
        blob.upload_from_filename(LOCAL_MODEL_PATH)

        print(f"✅ Pipeline uploaded to: gs://{BUCKET_NAME}/model/model.joblib")

        # This is the artifact URI you'll use for deployment
        ARTIFACT_URI = f"gs://{BUCKET_NAME}/model"
        print(f"📍 Artifact URI: {ARTIFACT_URI}")

    except FileNotFoundError:
        print(f"❌ File not found: {LOCAL_MODEL_PATH}")
        print("Make sure you trained and saved the model first!")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Please configure your project first")

## Step 3: Initialize Vertex AI


In [ ]:
# Initialize Vertex AI
if PROJECT_ID and PROJECT_ID not in ["YOUR_PROJECT_ID", "your-project-id-here"]:
    aiplatform.init(project=PROJECT_ID, location=REGION)
    print("✅ Vertex AI initialized")
else:
    print("❌ Please configure your project first")

## Step 4: Upload Model to Vertex AI Registry

Register your pipeline in Vertex AI Model Registry:

**Important:** Choose the correct scikit-learn container version!


In [ ]:
# Upload model to Vertex AI Model Registry
if 'ARTIFACT_URI' in locals():
    print("📦 Uploading model to Vertex AI Registry...")
    print("⏳ This may take 5-10 minutes...")

    try:
        model = aiplatform.Model.upload(
            display_name=MODEL_DISPLAY_NAME,
            artifact_uri=ARTIFACT_URI,
            serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest",
            # For different sklearn versions, use:
            # sklearn 1.0: "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-0:latest"
            # sklearn 1.2: "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest"
            # sklearn 1.3: "us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-3:latest"
        )

        print(f"✅ Model uploaded successfully!")
        print(f"Model resource name: {model.resource_name}")

    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Pipeline not uploaded to GCS yet")

## Step 5: Deploy to Endpoint

Use an existing endpoint or create a new one:


In [ ]:
# Get or create endpoint
if 'model' in locals():
    try:
        # Option A: Use existing endpoint
        print("🔍 Looking for existing endpoint...")
        endpoints = aiplatform.Endpoint.list(filter=f'display_name="{ENDPOINT_DISPLAY_NAME}"')

        if endpoints:
            endpoint = endpoints[0]
            print(f"✅ Found existing endpoint: {endpoint.display_name}")
            print(f"Resource name: {endpoint.resource_name}")
        else:
            # Option B: Create new endpoint if not found
            print("Creating new endpoint...")
            endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)
            print(f"✅ Endpoint created: {endpoint.display_name}")

        print(f"\n📍 Endpoint resource: {endpoint.resource_name}")

    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Model not uploaded yet")

In [ ]:
# Deploy model to endpoint
if 'endpoint' in locals() and 'model' in locals():
    print("🚀 Deploying model to endpoint...")
    print("⏳ This will take 10-15 minutes...")

    try:
        model.deploy(
            endpoint=endpoint,
            deployed_model_display_name=f"{MODEL_DISPLAY_NAME}-deployed",
            machine_type="n1-standard-2",  # You can use n1-standard-4 for more power
            min_replica_count=1,
            max_replica_count=1,
            traffic_percentage=100,  # Route 100% traffic to this version
        )

        print("\n🎉 Deployment completed successfully!")
        print(f"✅ Endpoint: {endpoint.display_name}")
        print(f"✅ Resource: {endpoint.resource_name}")
        print("\n💡 Now you can send RAW TEXT directly to get predictions!")

    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Endpoint or model not ready")

## Step 6: Test with Raw Text

Now you can send raw text directly without a vectorizer!


In [ ]:
# Test prediction with raw text
if 'endpoint' in locals():
    new_review = "I recently purchased this dress and I have to say, it exceeded my expectations!"

    print(f"Testing with review: '{new_review}'")
    print("\n...Calling endpoint...")

    try:
        # Send raw text directly - no vectorizer needed!
        response = endpoint.predict(instances=[new_review])

        print("\n--- Prediction Result ---")
        print(f"Prediction: {response.predictions[0]}")

        if response.predictions[0] == 1:
            print("📊 Classification: POSITIVE review ✓")
        else:
            print("📊 Classification: NEGATIVE review ✗")

    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Endpoint not ready")

## Step 7: Test with Multiple Reviews


In [ ]:
# Test with multiple reviews
if 'endpoint' in locals():
    test_reviews = [
        "I absolutely love this dress! The fit is perfect and the material is so soft.",
        "The shirt shrunk after one wash. Very disappointed with the quality.",
        "Amazing product! Highly recommend to everyone!",
        "Terrible quality. Waste of money."
    ]

    print("Testing multiple reviews...\n")

    try:
        for i, review in enumerate(test_reviews, 1):
            response = endpoint.predict(instances=[review])
            sentiment = "POSITIVE ✓" if response.predictions[0] == 1 else "NEGATIVE ✗"
            print(f"{i}. {review}")
            print(f"   → {sentiment}\n")
    except Exception as e:
        print(f"❌ Error: {e}")
else:
    print("❌ Endpoint not ready")

---

## 🎯 Success!

### You now have:

✅ Pipeline (vectorizer + model) deployed to Vertex AI  
✅ Ability to send raw text directly to endpoint  
✅ No need to load vectorizer locally  
✅ Production-ready deployment

### Useful Commands:

```python
# List all models
models = aiplatform.Model.list()
for m in models:
    print(m.display_name)

# List all endpoints
endpoints = aiplatform.Endpoint.list()
for ep in endpoints:
    print(ep.display_name)

# Undeploy old version
# endpoint.undeploy_all()

# Delete endpoint (careful!)
# endpoint.delete()
```

### Next Steps:

1. Test your endpoint using `08_Hit_Pipeline_Deployment/08_Pipeline_Deployment.ipynb`
2. Integrate with your application
3. Monitor performance in Google Cloud Console
4. Set up logging and monitoring
